## Object Tracking
***

Você teve uma introdução ao rastreamento de objeto sem vídeos com base na cor, o Color Tracking. Agora, veremos como detectar um objeto no vídeo usando não apenas cor, mas também seu movimento. Mas antes, vamos definir o que é Object Tracking

Object Tracking é o processo de localização de um objeto em movimento (ou objetos múltiplos) ao longo do tempo usando uma câmera. Tem uma variedade de usos, alguns dos quais são: interação homem-computador, segurança e vigilância, comunicação e compressão de vídeo, realidade aumentada, controle de tráfego, imagens médicas e edição de vídeo.

O Object Tracking pode ser um processo demorado devido à quantidade de dados contidos no vídeo. O processo pode ser ainda mais complexo se houver anecessidade de usar técnicas de reconhecimento de objetos para realizar o tracking, um problema ainda mais desafiador.

Em visão computacional e na aprendizagem de máquina, o tracking é um termo muito amplo que abrange ideias  conceitualmente  parecidas,  mas  tecnicamente  diferentes. Por exemplo, todas as técnicas abaixo são geralmente estudada sem Object Tracking:

* **Fluxo óptico denso (Dense Optical flow)**: Esses algoritmos ajudam a estimar o vetor de movimento de cada pixel em um frame de vídeo.


* **Fluxo óptico esparso (Sparse optical flow)**: Esses algoritmos, como o rastreador de características Kanade-Lucas-Tomashi (KLT), acompanham a localização de alguns pontos de recurso em uma imagem.


* **Filtragem de Kalman (Kalman Filtering)**: Um algoritmo de processamento de sinal muito popular usado para prever a localização de um objeto em movimento com base em informações de movimento anteriores. Uma das primeiras aplicações deste algoritmo foi a orientação de mísseis! O computador de bordo que guiou a descida do módulo lunar Apollo 11 para a lua tinha um filtro Kalman. Hoje é usado em carros autônomos.


* **Meanshift e Camshift**: São algoritmos para localizar o máximo de uma função de densidade. Eles também são usados para rastreamento.


* **Rastreadores de objetos únicos (Single object trackers)**: Nesta  classe  de rastreadores,  o  primeiro  frame  é  marcado  usando  um  retângulo  para  indicar  a localização do objeto que queremos rastrear. O objeto é então rastreado em frames subsequentes usando o algoritmo de rastreamento. Na maioria das aplicações da vida real, esses rastreadores são usados em conjunto com um detector de objetos.


* **Algoritmos de busca de rastreamento de objetos múltiplos (Multiple object track finding algorithms)**: Nos  casos  em  que  temos  um  detector  de  objeto  rápido,  faz sentido  detectar  vários  objetos  em  cada frame  e,  em  seguida,  executar um algoritmo de busca que identifique qual retângulo em um frame corresponde a um retângulo no próximo frame

***
### Tracking vs Detection
***

Ao realizar detecção facial com OpenCV, vimos que ele funciona em tempo real e você pode facilmente detectar o rosto em cada frame. Então, por que você precisa de tracking? Por que não aplicar as técnicas de detecção de objetos?

Vamos explorar os diferentes motivos pelos quais o tracking pode ser mais eficiente do que detectar objetos repetidamente em cada frame de um vídeo.

#### 1) Tracking é mais rápido que detecção 

Geralmente  os  algoritmos  de tracking são  mais rápidos do que os algoritmos de detecção. A razão é simples. Quando você está rastreando um objeto que foi detectado no frame anterior, você sabe muito sobre a aparência do  objeto. Você também conhece a localização no frame anterior e a direção e velocidade de seu movimento.

Então, no frame seguinte, você pode usar todas essas informações para prever a localização do objeto no frame e fazer uma pequena pesquisa em torno da localização esperada do objeto para localizar com precisão o objeto. Um bom algoritmo de rastreamento usará todas as informações sobre  o objeto até esse ponto, enquanto um algoritmo de detecção sempre começa a partir do zero.

Portanto, ao projetar um sistema eficiente, geralmente, uma detecção de objeto é executada em cada n-ésimo frame enquanto o algoritmo de rastreamento é empregado nos n-ésimo frame -1

Por que simplesmente não detectamos o objeto no primeiro frame e seguimos posteriormente? É verdade que o rastreamento se beneficia da informação extra que possui, mas você também pode perder o controle de um objeto quando ele fica atrás de um obstáculo por um longo período de tempo ou se ele se move tão rápido que o algoritmo de rastreamento não pode recuperar o atraso.

Também é comum que os algoritmos de rastreamento acumulem erros e a caixa delimitadora que rastreie o objeto se afasta lentamente do objeto que está rastreando. Para corrigir esses problemas com algoritmos de rastreamento, um algoritmo de detecção é executado de vez em quando. Os algoritmos de detecção são treinados em uma grande quantidade de exemplos do objeto. Eles, portanto, têm mais conhecimento sobre a classe geral do objeto. Por outro  lado, os algoritmos de rastreamento sabem mais sobre a instância específica da classe que estão rastreando.

#### 2) O Tracking pode ajudar quando a detecção falhar

Se você estiver executando um detector de rosto em um vídeo e o rosto da pessoa for obstruído por um objeto, o detector de rosto provavelmente falhará. Um bom algoritmo de rastreamento, por outro lado, irá lidar com algum nível de oclusão.

#### 3) O rastreamento preserva a identidade

A saída da detecção de objeto é uma matriz de retângulos que contém o objeto. No entanto, não há nenhuma identidade  anexada  ao  objeto. Por  exemplo, um detector que detecta pontos vermelhos produzirá retângulos correspondentes a todos os pontos detectados em um frame. No próximo frame, ele produzirá outra matriz de retângulos.

No primeiro frame, um ponto particular pode ser representado pelo retângulo no local 10 na matriz e no segundo frame, ele poderia estar na localização 17. Ao usar a detecção em um frame, não temos ideia de qual retângulo  corresponde a qual objeto. Por outro lado, o rastreamento fornece uma maneira de conectar literalmente os pontos!

O OpenCV 3 vem com uma nova API de rastreamento que contém implementações de muitos  algoritmos de rastreamento  de um único objeto. Existem 6 rastreadores diferentes disponíveis no OpenCV 3.2:

* Boosting
* MIL
* KCF
* TLD
* MedianFlow
* Goturn

***
### Algoritmos de Object Tracking e Detecção de Movimentos
***

Vamos investigar um pouco os diferentes algoritmos de tracking. O objetivo não é ter uma compreensão teórica profunda de cada rastreador, mas compreendê-los do ponto de vista prático.

Deixe-me começar por explicar primeiro alguns princípios gerais por trás do rastreamento. No rastreamento, nosso objetivo é encontrar um objeto no frame atual dado que rastreamos o objeto com sucesso em todos (ou quase todos) frames anteriores

Uma vez que rastreamos o objeto até o frame atual, sabemos como ele se moveu. Em outras palavras, conhecemos os  parâmetros do modelo de movimento. O modelo de movimento é apenas uma maneira elegante de dizer que você conhece  a localização e a velocidade (velocidade + direção do movimento) do objeto em frames anteriores. Se você não soubesse nada sobre o objeto, você poderia prever a nova localização com base no modelo de movimento atual, e você ficaria bem perto de onde está a nova localização do objeto.

Mas temos mais informações que apenas o movimento do objeto. Sabemos como o objeto se parece em cada um dos frames anteriores. Em outras palavras, podemos construir um modelo de aparência que codifique como é o objeto. Este modelo de aparência pode ser usado para  pesquisar em uma pequena região do local previsto pelo modelo de movimento para prever com precisão a localização do objeto.

O modelo de movimento prevê a localização aproximada do objeto. O modelo de aparência melhora esta estimativa  para fornecer uma estimativa mais precisa com base na aparência.

Se o objeto fosse muito simples e não mudasse a aparência, poderíamos usar um modelo simples como modelo de aparência e procurar esse modelo. No entanto, a vida real não é assim tão simples. A aparência de um objeto pode mudar dramaticamente. Para resolver esse  problema, em  muitos rastreadores modernos, esse modelo de aparência é  um classificador treinado de maneira on-line. Não entre em pânico! Deixe-me explicar em termos mais simples.

O trabalho do classificador é classificar uma região retangular de uma imagem como um objeto ou fundo. O classificador aceita um patch de imagem como entrada e retorna uma pontuação entre 0 e 1 para indicar a probabilidade de o patch de imagem conter o objeto. A pontuação é 0 quando é absolutamente certo de que o patch da imagem é o plano de fundo e 1 quando é absolutamente certo que o patch é o objeto.

Na aprendizagem de máquina, usamos a palavra "online" para se referir a algoritmos treinados online em tempo de execução. Um classificador off-line pode precisar de milhares de exemplos para treinar um classificador, mas um classificador online normalmente é treinado usando poucos exemplos em tempo de execução.

Um  classificador é treinado alimentando exemplos positivos (objeto) e negativos (background). Se você quer construir um classificador para detectar gatos, você o treina com milhares de imagens contendo gatos e milhares  de imagens que não contêm gatos. Desta forma, o classificador aprende a diferenciar o que é um gato e o que não é. Ao construir um classificador online, não temos o luxo de ter milhares de exemplos das classes positivas e negativas.

Agora, vejamos osdiferentes algoritmos de rastreamento que abordam esse problema de treinamento online.

***
### Boosting Tracker
***

Este rastreador é baseado em uma versão online do AdaBoost, o  algoritmo  que  o detector facial baseado em  cascata HAAR usa internamente.

Este classificador precisa ser treinado em tempo de execução com exemplos positivos e negativos do objeto. A caixa de delimitação inicial fornecida pelo usuário (ou por outro algoritmo de detecção de objetos) é tomada como o exemplo positivo para o objeto, e muitos patches de imagem fora da caixa delimitadora são tratados como o plano  de  fundo.

Dado um novo frame, o classificador é executado em cada pixel na região da localização anterior e a pontuação do classificador é gravada. A nova localização do objeto é aquela em que a pontuação é máxima. Então, agora temos um exemplo mais positivo para o classificador. À medida que mais frames chegam, o classificador é atualizado com esses dados adicionais.

***
### MIL Tracker
***

Este rastreador é semelhante à ideia para o rastreador BOOSTING descrito acima. A grande diferença é que, em vez de considerar apenas a localização atual do objeto como um exemplo positivo, ele procura em uma pequena região ao redor da localização atual para gerar vários exemplos positivos potenciais. Você pode estar pensando que é uma má ideia porque, na maioria desses exemplos "positivos", o objeto não está centrado.

É  aqui  que o  Multiple Instance Learning (MIL) mostra seu valor.  Em  MIL,  você  não especifica  exemplos  positivos  e  negativos,  mas  "sacos"  positivos  e  negativos.  A  coleção  de imagens no saco positivo não são todos exemplos positivos. Em vez disso, apenas uma imagem no saco positivo precisa ser um exemplo positivo! No nosso exemplo, um saco positivo contém o patch centrado na localização atual do objeto e também patches em uma pequena região ao redor. Mesmo que a localização atual do objeto rastreado não seja precisa, quando as amostras da vizinhança da localização atual forem colocadas no saco positivo, há uma boa chance de que este saco contenha pelo menos uma imagem na qual o objeto está bem centrado.

***
### KCF Kernelized Correlation Filters
***

KCF significa  filtros  de  correlação  Kernelizado sou Kernelized  Correlation  Filter.  Este rastreador baseia-se nas ideias apresentadas nos dois rastreadores anteriores. Este rastreador utiliza o fato de que as múltiplas amostras positivas usadas no rastreador MIL têm grandes regiões  sobrepostas.  Esta  sobreposição  de  dados  leva  a  algumas  boas  propriedades matemáticas que são exploradas por esse rastreador para tornar o rastreamento mais rápido e preciso ao mesmo tempo.

* **Prós**:  Precisão  e  velocidade  são  melhores  do  que  MIL  e  informa  falhas  de  rastreamento melhores que BOOSTING e MIL. Se você estiver usando o OpenCV 3.1 ou superior, este é o melhor algoritmo para tracking de objetos.


* **Contras**: Não se recupera da oclusão total. Não implementado no OpenCV 3.0.

#### Como o KCF funciona?

Dado  o  conjunto  inicial  de  pontos, o rastreador  tenta  calcular o  movimento  desses pontos,  observando  a  direção  da  mudança  no  próximo frame. Em  cada frame consecutivo, tentamos procurar o mesmo conjunto de pontos na vizinhança. Uma vez identificadas as novas posições desses pontos, podemos mover a caixa delimitadora sobre o novo conjunto de pontos. Há matemática envolvida em fazer a busca mais rápida e eficiente

***
### TLD Tracker
***

TLD significa Rastreamento, aprendizado e detecção (Tracking, learning and detection). Como o nome sugere, este rastreador decompõe a tarefa de rastreamento a longo prazo em três componentes - rastreamento, aprendizado e detecção.

O rastreador segue o objeto frame a frame. O detector localiza todas as aparências que foram observadas e corrige o rastreador, se necessário. A aprendizagem calcula os erros do detector e atualiza-o para evitar esses erros no futuro. A saída  desse  rastreador  tende  a  saltar  um  pouco.  Por  exemplo,  se  você  está rastreando  um  pedestre  e  há  outros  pedestres  na  cena,  esse rastreador  às  vezes  pode acompanhar temporariamente um pedestre diferente do que você pretende acompanhar.

No lado positivo, este Tracker parece acompanhar um objeto em uma escala maior, mesmo em movimento e com oclusão. Se você tem uma sequênciade vídeo onde o objeto está escondido atrás de outro objeto, esse rastreador pode ser uma boa opção.

* **Prós**:  funciona  melhor  em  oclusão  em  vários frames.  Além  disso,  acompanha  melhor  as mudanças de escala.


* **Contras**: muitos falsos positivos tornando-o quase inutilizável.

***
### MEDIANFLOW Tracker
***

Internamente, este tracker rastreia o objeto em ambas as direções para frente e para trás no tempo e mede as discrepâncias entre essas duas trajetórias. A minimização desse erro do ForwardBackward  permite  que eles  detectem com  segurança falhas  de  rastreamento  e selecionem trajetórias confiáveis em sequências de vídeo.

Este  rastreador  funciona  melhor  quando  o  movimento  é  previsível  e  pequeno.  Ao contrário,  outros  rastreadores  que  continuam  mesmo  quando  o  rastreamento  falha claramente, esse rastreador sabe quando o rastreamento falhou.

* **Prós**: excelente relatório de falhas de rastreamento. Funciona muito bem quando o movimento é previsível e não há oclusão.


* **Contras**: falha sob grande movimento

***
### GOTURN Tracker
***

Entretodos  os  algoritmos  de  rastreamento,  esse  é  o  único  baseado  na  rede neural convolucional (CNN). É também o único que usa um modelo treinado offline, e com isso é mais rápido que outros rastreadores. A partir da documentação OpenCV, sabemos que é "robusto para mudanças de ponto de vista, mudanças de iluminação e deformações". Mas não lida com a oclusão muito bem. Estudaremos as CNNs mais a frente aqui mesmo no curso.